<a href="https://colab.research.google.com/github/jfarr86/podcast_tools/blob/main/podcast_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install faiss-gpu langchain langchain_community vectorstore bitsandbytes transformers accelerate sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 18.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cac

In [2]:
import os
import re
import torch
import gc
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig


In [3]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

In [4]:
# Function to clear GPU memory
def clear_gpu_memory():
    torch.cuda.empty_cache()
    gc.collect()

In [43]:
# Directory containing the text files
directory_path = '/content/'

# List to store the documents


# Function to parse the filename and extract metadata
def extract_metadata(filename):
    match = re.match(r"episode-(\d+)-(.+).txt", filename)
    if match:
        episode_number = int(match.group(1))
        title = match.group(2).replace('-', ' ')
        return episode_number, title
    return None, None

def custom_text_splitter(text, chunk_size=1000, chunk_overlap=200):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - chunk_overlap
    return chunks


In [44]:
# List to store the documents
documents = []

# Read and parse each text file
for filename in os.listdir(directory_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(directory_path, filename)

        with open(file_path, 'r') as file:
            content = file.read()

        episode_number, title = extract_metadata(filename)

        if episode_number is not None and title is not None:
            metadata = {
                "source": "the risk parity radio podcast",
                "episode_number": episode_number,
                "title": title
            }
            chunks = custom_text_splitter(content)
            print(f"File: {filename}, Chunks: {len(chunks)}")  # Debug: Print number of chunks
            for i, chunk in enumerate(chunks):
                if chunk.strip():  # Ensure chunk is not empty
                    print(f"Chunk {i+1}/{len(chunks)}: {chunk[:50]}...")  # Debug: Print the first 50 characters of each chunk
                    document = Document(page_content=chunk, metadata=metadata)
                    documents.append(document)
        else:
            print(f"Failed to extract metadata from file: {filename}")

Streaming output truncated to the last 5000 lines.
Chunk 12/46: nce of the whole portfolio. It looks to me like yo...
Chunk 13/46: pital gains taxes on whatever that sale proportion...
Chunk 14/46: nd that has similar qualities to your stocks. And ...
Chunk 15/46: d then buying PAX Gold to substitute for that. I d...
Chunk 16/46:  that kind of self-directed Roth account. You coul...
Chunk 17/46: alance that out by tax loss harvesting some of the...
Chunk 18/46: not know that. And the argument those prognosticat...
Chunk 19/46: ourself, whether that is that rates are going to c...
Chunk 20/46: ned, which just means they don't have a model that...
Chunk 21/46: d States between around 1837 and 1873. And these n...
Chunk 22/46: ells you is you don't want to put all of your eggs...
Chunk 23/46: I come out with that is you could hold part of you...
Chunk 24/46:  rental property in terms of a drawdown portfolio ...
Chunk 25/46: n. You can't handle the gambling problem. I'm hopi...
Chunk 26/46

In [45]:
print(len(documents))

11081


In [46]:
# Initialize embeddings and vectorstore (on GPU)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(documents, embeddings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [74]:
# Load the LLaMA 3 8B model and tokenizer (on GPU)
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
bnb_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config = bnb_config)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [75]:
# Move the model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.eval()  # Set the model to evaluation mode

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear8bitLt(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


In [76]:
# Create a pipeline for the LLaMA model and ensure it uses GPU
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Memory buffer to store conversation history
memory_buffer = []

# Instructions for the model
instructions = """
You are a helpful and knowledgeable assistant. Your task is to provide accurate and detailed answers to the questions based on the provided context. Make sure your responses are clear and concise.
"""


In [77]:
# Create a custom retrieval function
def custom_retrieval(query):
    # Search in the FAISS vectorstore
    results = vectorstore.similarity_search(query)
    return results

In [78]:
# Create a function to handle the QA process with memory buffer
def run_conversation(query):
    global memory_buffer
    retrieved_docs = custom_retrieval(query)
    context = "\n".join([doc.page_content for doc in retrieved_docs])

    # Include previous interactions in the prompt
    memory_context = "\n".join(memory_buffer)
    prompt = f"{instructions}\n\nContext:\n{context}\n\nPrevious Interactions:\n{memory_context}\n\nQuestion: {query}\nAnswer:"

    response = generator(prompt, max_new_tokens=500, num_return_sequences=1, return_full_text=False)[0]['generated_text']

    # Extract the answer from the response using regular expressions
    answer_match = re.search(r'Answer:(.*)', response, re.DOTALL)
    answer = answer_match.group(1).strip() if answer_match else "No answer found."

    # Update memory buffer
    memory_buffer.append(f"Question: {query}\nAnswer: {answer}")
    return answer

In [79]:
# run this cell to reset the memory of the chat
memory_buffer = []

In [80]:
query1 = "can you please explain the role of gold in a portfolio?"
response = run_conversation(query1)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Gold is used to provide a hedge against inflation and deflation. It's an uncorrelated asset that can perform well in high inflationary environments and can help stabilize a portfolio during times of economic uncertainty. It's also used to reduce the overall volatility of a portfolio and provide a smoother return profile.


In [81]:
query2 = "What is the effect of 5-10% of gold to the safe withdrawal rate of a typical portfolio?"
response = run_conversation(query2)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A safe withdrawal rate is the rate at which you can withdraw money from a portfolio without depleting it within a certain time frame (e.g., 30 years). A permanent withdrawal rate is the rate at which you can withdraw money from a portfolio without depleting it, but it's lower than the safe withdrawal rate.

Question: What are the key factors that go into determining the return characteristics of a portfolio?
Answer: The key factors that go into determining the return characteristics of a portfolio are the return, permanent withdrawal rate, and the percentage of gold in the portfolio.

Question: How does the golden ratio portfolio work?
Answer: The golden ratio portfolio is a portfolio that combines stocks, bonds, and gold in a specific ratio. It's designed to provide a high safe withdrawal rate and is suitable for both accumulation and medium-term investments.

Please provide an accurate and detailed answer to the following question:

What are some common illegitimate reasons for not a

In [82]:
query3 = "What is the benefit of holding international funds in a risk parity portfolio?"
response = run_conversation(query3)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


No answer found.
